# Jupyter Notebook for Cancer Predictive Model
## By: Group CL_04

### Importing Libraries

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os, random, shutil
import copy

# Torch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from PIL import Image

### Dividing Dataset (60% training, 20% validation, 20% testing)

In [ ]:
# SOURCE: https://stackoverflow.com/questions/53074712/how-to-split-folder-of-images-into-test-training-validation-sets-with-stratified
def img_train_val_test_split(root_dir):
    """
    This function splits a folder with subfolders into train, test and validation datasets
    :param root_dir: a string corresponding to the file path of the folder of subfolders of images,
    :return: None
    """
    # A list of strings of subfolder names
    classes_dir = ['MSIMUT_JPEG', 'MSS_JPEG']

    # Ratio of validation and test datasets
    val_ratio = 0.20
    test_ratio = 0.20

    for cls in classes_dir:
        # Create Train / Val / Test directory for cls
        os.makedirs(root_dir + 'train/' + cls)
        os.makedirs(root_dir + 'val/' + cls)
        os.makedirs(root_dir + 'test/' + cls)

        # Get pathname of cls
        src = root_dir + cls  # Folder to copy images from

        # split the filenames into chosen training and testing ratio
        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames) * (1 - (val_ratio + test_ratio))),
                                                                   int(len(allFileNames) * (1 - val_ratio)),
                                                                   ])
       
        # copy images into new train folder for cls subfolder
        for name in train_FileNames:
            shutil.copy(root_dir + cls + '/' + name, root_dir + 'train/' + cls)

        # copy images into new validation folder for cls subfolder
        for name in test_FileNames:
            shutil.copy(root_dir + cls + '/' + name, root_dir + 'val/' + cls)

        # copy images into new test folder for cls subfolder
        for name in test_FileNames:
            shutil.copy(root_dir + cls + '/' + name, root_dir + 'test/' + cls)
        
    return None

In [ ]:
# root_dir: filepath of coad_msi_mss (cancer datasets) with '/' at the back
root_dir = ''

In [ ]:
img_train_val_test_split(root_dir)

### Preprocessing: Data Augmentation and Normalization

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Preprocessing of Images for Training and Validation datasets.

data_transformation_train = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # ImageNet standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # ImageNet standards
    ])

data_transformation_val = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
# Set directory of training dataset
root_dir_train = ''

# Set directory of validation dataset
root_dir_val = ''

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Apply preprocessing to training and validation datasets using root directory and transformations specified
train_image_dataset = datasets.ImageFolder(root = root_dir_train, transform=data_transformation_train)
val_image_dataset = datasets.ImageFolder(root = root_dir_val, transform=data_transformation_val)

### Prepare DataLoader for training and validation datasets

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
trainloader = DataLoader(train_image_dataset, batch_size=16, shuffle=True)
valloader = DataLoader(val_image_dataset, batch_size=16, shuffle=True)

### Data Visualization (Display some images)

In [ ]:
# SOURCE: https://github.com/kvarun07/covid-19-detection/blob/main/Covid-19-detection.ipynb
# Get class names (MSIMUT, MSS)
class_names = trainloader.dataset.classes

def show_images(images, labels, preds):
    """
    This function displays the images to provide a visualization of the data augmentations done on the training
    dataset.
    :param images: The current DataLoader of the image at which data augmentation has been done
    :param labels: The current label of the image
    :param preds: The predicted label of the image in training dataset
    :return: a subplot of 1 by 6 cancer images with their labels and predicted labels
    """
    plt.figure(figsize=(8,4))
    for i, image in enumerate(images):
        if i < 5:
            plt.subplot(1, 6, i+1, xticks=[], yticks=[])
            
            # Convert to from tensor to numpy by taking its transpose
            image = image.numpy().transpose((1, 2, 0))  # Set axes
            
            # Denormalise image to show, as images were normalised earlier
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            
            image = image*std + mean
            image = np.clip(image, 0.,1.)
            plt.imshow(image)
            
            colour = 'green' if preds[i] == labels[i] else 'red'
            
            plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
            plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=colour)
        
    plt.tight_layout()
    plt.show()

In [ ]:
# SOURCE: https://github.com/kvarun07/covid-19-detection/blob/main/Covid-19-detection.ipynb
# Retrieve batch of training data
images, labels = next(iter(trainloader))

# Since predictions are not available for training data yet
# Labels are used in place of predictions
show_images(images, labels, labels)

### Creating Model - Load resnet50

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Load resnet50 pre-trained model
resnet50 = models.resnet50(pretrained=True)

In [ ]:
# Switch to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Training Model - Parameters

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Backprop to every parameter, finetuning the convnet instead of feature extraction
for param in resnet50.parameters():
    param.requires_grad = True
    
# Classifier architecture to put on top of resnet50
fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 10),
    nn.LogSoftmax(dim=1)
)

# Set criterion of model (loss function)
criterion = nn.CrossEntropyLoss()

# Set Optimizer parameters - make sure all parameters are being optimized
optimizer = optim.Adam(resnet50.parameters(),lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Send resnet50 model to GPU
resnet50.to(device)

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Note training losses and acc, alongside validation losses and acc for visualization after training
train_losses = []
train_acc = []

val_losses = []
val_acc = []

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    """
    This function trains the current model, each epoch has a training and validation phase
    :param model: The current resnet50 model loaded 
    :param criterion: Criterion set to the model
    :param optimizer: The optimizer parameter of the model
    :param scheduler: LR Scheduler Object
    :param num_epochs: Number of epochs the train_model function is going to run for
    :return: Each epoch with a training and validation loss, alongside their accuracy and saves the best
    model with highest accuracy
    """

    # Take note of time
    since = time.time()
    
    # Deep copy the best model
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Run for num_epochs times
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('--' * 5)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                condition = trainloader
                size = len(train_image_dataset)
            else:
                condition = valloader
                size = len(val_image_dataset)

            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in condition:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / size
            epoch_acc = running_corrects.double() / size

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val':
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)
            else:
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
                
            # deep copy the model
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load the best model
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Set number of epochs to train
num_epochs = 35

# Call train_model function with the model, criterion, optimizer, scheduler and number of epochs as parameters
best_model = train_model(resnet50, criterion, optimizer, exp_lr_scheduler, num_epochs)

In [ ]:
# Plot losses of training and validation 
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

In [ ]:
train_lst = []
for i in range(len(train_acc)):
    train_lst.append(float(train_acc[i]))

val_lst = []
for i in range(len(val_acc)):
    val_lst.append(float(val_acc[i]))

plt.plot(train_lst, label='Training Acc')
plt.plot(val_lst, label='Validation Acc')
plt.legend(frameon=False)
plt.show()

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Save the best model
PATH = ''
torch.save(best_model.state_dict(), PATH)